In [41]:
import pandas as pd
import numpy as np

### Model Imports
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer




In [7]:
df =pd.read_csv('data/train.csv')

In [8]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [9]:
### drop columns which do not contribute in the analysis
df = df.drop(columns=['Name','Ticket','Cabin'])


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    object 
 4   Age          714 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Fare         891 non-null    float64
 8   Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(2)
memory usage: 62.8+ KB


In [12]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [14]:
### Creating 2 new columns
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
df['IsAlone'] = df['FamilySize'].apply(lambda x: 1 if x == 1 else 0)

In [20]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Fare             0
Embarked         2
FamilySize       0
IsAlone          0
dtype: int64

In [25]:
X_train = df.drop(columns='Survived')
y_train = df['Survived']

In [30]:
from sklearn.impute import SimpleImputer
num_features = X_train.select_dtypes(exclude="object").columns
cat_features = X_train.select_dtypes(include="object").columns

X_train[num_features] = SimpleImputer(strategy='mean').fit_transform(X_train[num_features])
X_train[cat_features] = SimpleImputer(strategy='most_frequent').fit_transform(X_train[cat_features])



In [31]:
num_transformer = StandardScaler()
cat_transformer = OneHotEncoder()

transformation = ColumnTransformer(
    [
    ("OneHot Encoder", cat_transformer, cat_features),
    ("Standard Scaler", num_transformer, num_features)
    ]
)

In [32]:
X_train = transformation.fit_transform(X_train)

In [17]:
df1 = pd.read_csv('data/test.csv')

In [18]:
### drop columns which do not contribute in the analysis
df1 = df1.drop(columns=['Name','Ticket','Cabin'])


In [19]:
### Creating 2 new columns
df1['FamilySize'] = df1['SibSp'] + df1['Parch'] + 1
df1['IsAlone'] = df1['FamilySize'].apply(lambda x: 1 if x == 1 else 0)

In [33]:
df1.isna().sum()

PassengerId     0
Pclass          0
Sex             0
Age            86
SibSp           0
Parch           0
Fare            1
Embarked        0
FamilySize      0
IsAlone         0
dtype: int64

In [34]:
X_test = df.drop(columns='Survived')
y_test = df['Survived']

In [35]:
num_features = X_test.select_dtypes(exclude="object").columns
cat_features = X_test.select_dtypes(include="object").columns

X_test[num_features] = SimpleImputer(strategy='mean').fit_transform(X_test[num_features])
X_test[cat_features] = SimpleImputer(strategy='most_frequent').fit_transform(X_test[cat_features])



In [36]:
num_transformer = StandardScaler()
cat_transformer = OneHotEncoder()

transformation = ColumnTransformer(
    [
    ("OneHot Encoder", cat_transformer, cat_features),
    ("Standard Scaler", num_transformer, num_features)
    ]
)

In [37]:
X_test = transformation.fit_transform(X_test)

In [ ]:
def evaluate_model(true, predicted):
    f1score = f1_score(true, predicted)
    prescision = precision_score(true, predicted)
    recall = recall_score(true, predicted)
    # roc_auc = roc_auc_score(true, predicted)
    confusion = confusion_matrix(true, predicted)
    accuracy = accuracy_score(true, predicted)

    return f1score,  prescision, recall, confusion, accuracy

In [46]:
models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest Classifier" : RandomForestClassifier(),
    "AdaBoost Classifier": AdaBoostClassifier(),
    "CatBoost Classifer": CatBoostClassifier(verbose=1),
    "XGB Classifier": XGBClassifier(),
    "Decission Tree": DecisionTreeClassifier(),
    "KNeighbors Classifier": KNeighborsClassifier(),
    "Gaussian NB" : GaussianNB()
}

model_list = []
accuracies = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    model_train_f1, model_train_precision, model_train_recall, model_train_confusion, model_train_accuracy = evaluate_model(y_train, y_train_pred)
    model_test_f1, model_test_precision, model_test_recall, model_test_confusion, model_test_accuracy = evaluate_model(y_test, y_test_pred)


    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Performance on Training Data')
    print("- F1 Score : {:.4f}".format(model_train_f1))
    print("- Precision Score : {:.4f}".format(model_train_precision))
    print("- Recall Score : {:.4f}".format(model_train_recall))
    print("- Confusion Matrix : ", (model_train_confusion))
    print("- Accuracy: {:.4f}".format(model_train_accuracy))

    print('-----------------------------')

    print("Model Performance on Test Data")
    print("- F1 Score : {:.4f}".format(model_test_f1))
    print("- Precision Score : {:.4f}".format(model_test_precision))
    print("- Recall Score : {:.4f}".format(model_test_recall))
    print("- Confusion Matrix :",(model_test_confusion))
    print("- Accuracy: {:.4f}".format(model_test_accuracy))

    print('='*35)
    print('\n')

Logistic Regression
Model Performance on Training Data
- F1 Score : 0.7372
- Precision Score : 0.7625
- Recall Score : 0.7135
- Confusion Matrix :  [[473  76]
 [ 98 244]]
- Accuracy: 0.8047
-----------------------------
Model Performance on Test Data
- F1 Score : 0.7372
- Precision Score : 0.7625
- Recall Score : 0.7135
- Confusion Matrix : [[473  76]
 [ 98 244]]
- Accuracy: 0.8047


Random Forest Classifier
Model Performance on Training Data
- F1 Score : 1.0000
- Precision Score : 1.0000
- Recall Score : 1.0000
- Confusion Matrix :  [[549   0]
 [  0 342]]
- Accuracy: 1.0000
-----------------------------
Model Performance on Test Data
- F1 Score : 1.0000
- Precision Score : 1.0000
- Recall Score : 1.0000
- Confusion Matrix : [[549   0]
 [  0 342]]
- Accuracy: 1.0000


AdaBoost Classifier
Model Performance on Training Data
- F1 Score : 0.7727
- Precision Score : 0.7855
- Recall Score : 0.7602
- Confusion Matrix :  [[478  71]
 [ 82 260]]
- Accuracy: 0.8283
-----------------------------
M